# Data-Gathering

In [1]:
import numpy as np
import pandas as pd
import wmfdata as wmf

pd.options.display.max_columns = None
from IPython.display import clear_output

import importlib
import warnings

import data_functions as dtf
import formatting_functions as ftf

You are using Wmfdata v2.0.0, but v2.0.1 is available.

To update, run `pip install --upgrade git+https://github.com/wikimedia/wmfdata-python.git@release`.

To see the changes, refer to https://github.com/wikimedia/wmfdata-python/blob/release/CHANGELOG.md.


In [2]:
importlib.reload(dtf)
importlib.reload(ftf)

<module 'formatting_functions' from '/srv/home/kcv-wikimf/gitref/content-translation-deletion-stats/formatting_functions.py'>

## spark_session

In [3]:
spark_session = wmf.spark.get_active_session()

if type(spark_session) != type(None):
    spark_session.stop()
else:
    print('no active session')

no active session


In [4]:
spark_session = wmf.spark.create_custom_session(
    master="yarn",
    app_name='cx-deletion-stats',
    spark_config={
        "spark.driver.memory": "4g",
        "spark.dynamicAllocation.maxExecutors": 64,
        "spark.executor.memory": "16g",
        "spark.executor.cores": 4,
        "spark.sql.shuffle.partitions": 256,
        "spark.driver.maxResultSize": "2g"
        
    }
)

clear_output()

spark_session

In [5]:
spark_session.sparkContext.setLogLevel("ERROR")

## run query

In [ ]:
%%time

for y in [2020, 2021, 2022]:
        for q in [f'Q{_}' for _ in range(1, 4+1)]:
            if (y == 2020) & (q in ['Q1', 'Q2']):
                pass
            else:
                quarter = dtf.generate_quarters(y)[q]
                quarterly_stats = dtf.query_deletion_stats(quarter)
                overall_deletion_rate = dtf.overall_deletion_pct(quarterly_stats)
                print(f'From {quarter["start_dt"]} to {quarter["end_dt"]} - deletion rate of CX created articles: {overall_deletion_rate["deleted_cx_pct"]}%;\
                        deletion rate of non-CX created articles: {overall_deletion_rate["deleted_non_cx_pct"]}%\n')

In [ ]:
%%time

warnings.filterwarnings('ignore')

deletion_stats_currq_all = dtf.query_deletion_stats(currq_dates)
deletion_stats_prevq_all = dtf.query_deletion_stats(prevq_dates)